# Planejamento Estratégico de Expansão de Capacidade
## Estudo de Caso: Ciclo Motores S/A

**Autora:** Amanda Dallepiane de Souza
**Matrícula:** 23200478

---

### 1. Contextualização
O objetivo deste relatório é determinar o plano ótimo de expansão da capacidade produtiva da **Ciclo Motores S/A** para os próximos 10 anos.

O problema é modelado considerando:
- **Demanda Estocástica:** A demanda futura segue uma distribuição normal com média e desvio padrão variáveis ano a ano.
- **Decisões Sequenciais:** A decisão tomada em um ano afeta a capacidade disponível no ano seguinte.
- **Maximização de Valor:** O objetivo é maximizar o Valor Presente Líquido (VPL) dos fluxos de caixa, descontados a uma taxa de 9,3% a.a.

A metodologia utilizada foi a **Programação Dinâmica Estocástica**.

### a) Formulação do Modelo de Decisão

O problema foi formulado como um processo de decisão sequencial (Programação Dinâmica).

**Definições:**
* **Estado ($S_t$):** Capacidade instalada atual no início do ano $t$.
* **Decisão ($x_t$):** Quantidade de capacidade a adicionar (0, 50k, 100k, 150k, 200k).
* **Estágio ($t$):** Anos de 1 a 10.

**Equação de Recorrência (Equação de Bellman):**

Para cada ano $t$, buscamos maximizar o lucro imediato menos o custo de investimento, somado ao valor futuro esperado das decisões ótimas:

$$V_t(S_t) = \max_{x_t} \left\{ \text{LucroOp}(S_t) - \text{Investimento}(x_t) + \frac{1}{1+i} \cdot V_{t+1}(S_t + x_t) \right\}$$

Onde:
* $\text{LucroOp}(S_t)$ é o lucro esperado considerando a incerteza da demanda (perda por falta de estoque ou excesso de capacidade).
* $i$ é a taxa de desconto (9,3%).

In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt # Opcional, para gráficos

# Configuração de visualização
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# --- DADOS DE ENTRADA ---

# Demanda: {Ano: {Media, Desvio Padrao}}
dados_demanda = {
    1:  {'mu': 140000, 'sigma': 13300},
    2:  {'mu': 182000, 'sigma': 17300},
    3:  {'mu': 231100, 'sigma': 22000},
    4:  {'mu': 281100, 'sigma': 26700},
    5:  {'mu': 323600, 'sigma': 30700},
    6:  {'mu': 352900, 'sigma': 33500},
    7:  {'mu': 372100, 'sigma': 35400},
    8:  {'mu': 384300, 'sigma': 36500},
    9:  {'mu': 391800, 'sigma': 37200},
    10: {'mu': 396400, 'sigma': 37700}
}

# Opções de Expansão e Custos
opcoes_expansao = {
    0: 0,
    50000: 100_000_000,
    100000: 190_000_000,
    150000: 270_000_000,
    200000: 350_000_000
}

# Parâmetros Financeiros e Operacionais
lucro_por_unidade = 1088
capacidade_inicial = 150000
taxa_desconto = 0.093

#### Função de Venda Esperada

Como a demanda é uma variável aleatória $D \sim N(\mu, \sigma^2)$, a venda efetiva é limitada pela capacidade instalada $K$. Ou seja, $\text{Venda} = \min(D, K)$.

A função abaixo calcula a esperança matemática dessa venda: $E[\min(D, K)]$.

In [6]:
def calcular_venda_esperada(capacidade, mu, sigma):
    """
    Calcula a venda esperada dada uma distribuição normal e um limite de capacidade.
    Utiliza a perda parcial da distribuição normal.
    """
    z = (capacidade - mu) / sigma
    
    # Termo 1: Probabilidade de vender menos que a capacidade
    # Termo 2: Probabilidade de a demanda estourar a capacidade (vende o teto)
    venda_esperada = mu * norm.cdf(z) - sigma * norm.pdf(z) + capacidade * (1 - norm.cdf(z))
    return venda_esperada

In [7]:
# --- ALGORITMO DE PROGRAMAÇÃO DINÂMICA (Backward Induction) ---

memo = {}
estados_possiveis = [150000 + i*50000 for i in range(15)] # Grid de estados suficientes

# Iteração do Ano 10 até o Ano 1
for ano in range(10, 0, -1):
    memo[ano] = {}
    mu = dados_demanda[ano]['mu']
    sigma = dados_demanda[ano]['sigma']
    
    for cap_atual in estados_possiveis:
        melhor_vpl = -np.inf
        melhor_decisao = None
        
        # Fluxo Operacional do ano corrente
        vendas = calcular_venda_esperada(cap_atual, mu, sigma)
        fco = vendas * lucro_por_unidade
        
        # Avaliar decisões de investimento para o próximo ano
        for expansao, investimento in opcoes_expansao.items():
            cap_proximo = cap_atual + expansao
            
            # Valor de Continuação (Futuro)
            if ano == 10:
                # Perpetuidade (Assumindo estabilidade no ano 11 em diante)
                vendas_term = calcular_venda_esperada(cap_proximo, mu, sigma)
                fluxo_perpetuo = vendas_term * lucro_por_unidade
                valor_futuro = fluxo_perpetuo / taxa_desconto
            else:
                # Busca valor no ano seguinte (se dentro do grid)
                if cap_proximo in memo[ano+1]:
                    valor_futuro = memo[ano+1][cap_proximo][0]
                else:
                    valor_futuro = 0 

            # Cálculo do VPL da decisão
            vpl_decisao = fco - investimento + (valor_futuro / (1 + taxa_desconto))
            
            if vpl_decisao > melhor_vpl:
                melhor_vpl = vpl_decisao
                melhor_decisao = expansao
        
        memo[ano][cap_atual] = (melhor_vpl, melhor_decisao)

# --- RECONSTRUÇÃO DA SOLUÇÃO (Forward Pass) ---
plano = []
cap_atual = capacidade_inicial
vpl_total = memo[1][cap_atual][0]

for ano in range(1, 11):
    decisao = memo[ano][cap_atual][1]
    custo = opcoes_expansao[decisao]
    
    plano.append({
        "Ano": ano,
        "Capacidade Inicial": cap_atual,
        "Decisão de Expansão": decisao,
        "Investimento (US$)": custo,
        "Capacidade Final": cap_atual + decisao
    })
    
    cap_atual += decisao

df_plano = pd.DataFrame(plano)

### b) Plano Ótimo de Expansão

Abaixo apresenta-se a trajetória ótima de expansão. O modelo sugere investimentos agressivos nos primeiros anos para acompanhar o crescimento da demanda média, aproveitando a economia de escala antes que a demanda se estabilize.

In [8]:
# Formatação para exibição monetária
format_mapping = {
    'Capacidade Inicial': '{:,.0f}',
    'Decisão de Expansão': '{:,.0f}',
    'Investimento (US$)': '${:,.2f}',
    'Capacidade Final': '{:,.0f}'
}

df_plano.style.format(format_mapping).hide(axis="index")

Ano,Capacidade Inicial,Decisão de Expansão,Investimento (US$),Capacidade Final
1,"150,000","100,000","$190,000,000.00","250,000"
2,"250,000",0,$0.00,"250,000"
3,"250,000","150,000","$270,000,000.00","400,000"
4,"400,000",0,$0.00,"400,000"
5,"400,000",0,$0.00,"400,000"
6,"400,000",0,$0.00,"400,000"
7,"400,000",0,$0.00,"400,000"
8,"400,000","50,000","$100,000,000.00","450,000"
9,"450,000",0,$0.00,"450,000"
10,"450,000",0,$0.00,"450,000"


### c) Determinação do Valor da Ciclo Motores S/A

O valor econômico da empresa, calculado através do Valor Presente Líquido (VPL) de todos os fluxos de caixa futuros otimizados (incluindo o valor terminal na perpetuidade), é:

In [9]:
print(f"Valor da Empresa (VPL): US$ {vpl_total:,.2f}")

Valor da Empresa (VPL): US$ 3,552,064,806.14


### d) Análise da Aplicabilidade do Modelo

O modelo proposto apresenta robustez e limitações que devem ser consideradas pela diretoria:

**Pontos Fortes (Aplicabilidade):**
1.  **Tratamento de Risco:** Ao utilizar a integral da distribuição normal (e não apenas a média), o modelo penaliza cenários onde a capacidade é excessiva comparada à demanda real, evitando "custos afundados" desnecessários.
2.  **Visão de Longo Prazo:** A inclusão da perpetuidade garante que as decisões tomadas nos últimos anos do projeto (ex: ano 9 ou 10) não sejam míopes, considerando o valor gerado pós-horizonte de planejamento.

**Limitações e Pontos de Atenção:**
1.  **Premissas Econômicas Estáticas:** O modelo assume que o preço de venda (US$ 1.088) e os custos de expansão permanecem constantes por 10 anos. Na realidade, a inflação e custos de matéria-prima variam.
2.  **Rigidez Temporal:** Assume-se que a expansão leva exatamente 1 ano. Atrasos em obras poderiam afetar drasticamente o VPL, sugerindo a necessidade de uma análise de sensibilidade quanto ao *time-to-market*.

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# --- CONFIGURAÇÃO DOS DADOS ---

# Parâmetros Econômicos
TMA = 0.093  # Taxa de Mínima Atratividade (9,3%)
LUCRO_POR_MIL_UNID = 1.088  # US$ 1088 por unidade = US$ 1.088 Milhões por 1000 unidades
CAPACIDADE_INICIAL = 150  # mil unidades

# Opções de Expansão (Ações k)
# Chave: Aumento de capacidade, Valor: Custo em Milhões
OPCOES_EXPANSAO = {
    0: 0,
    50: 100,
    100: 190,
    150: 270,
    200: 350
}

# Dados da Demanda (Média e Desvio Padrão por Ano)
demandas = pd.DataFrame({
    'Ano': range(1, 11),
    'Media': [140.0, 182.0, 231.1, 281.1, 323.6, 352.9, 372.1, 384.3, 391.8, 396.4],
    'StdDev': [13.3, 17.3, 22.0, 26.7, 30.7, 33.5, 35.4, 36.5, 37.2, 37.7]
})

# --- FUNÇÕES AUXILIARES ---

def calcular_vendas_esperadas(capacidade, media, desvio):
    """
    Calcula E[min(Demanda, Capacidade)] para uma distribuição normal.
    Isso é necessário porque se a demanda for maior que a capacidade,
    vendemos apenas a capacidade. Se for menor, vendemos a demanda.
    """
    if desvio == 0:
        return min(capacidade, media)
    
    # Z-score da capacidade
    z = (capacidade - media) / desvio
    
    # Termo 1: Parte da demanda que é menor que a capacidade
    term1 = media * norm.cdf(z) - desvio * norm.pdf(z)
    
    # Termo 2: Parte onde a demanda excede a capacidade (limitada pela capacidade)
    term2 = capacidade * (1 - norm.cdf(z))
    
    return term1 + term2

def calcular_lucro_operacional(capacidade, ano_idx):
    """Calcula o lucro operacional esperado para um dado ano e capacidade."""
    dados_ano = demandas.iloc[ano_idx]
    vendas_esperadas = calcular_vendas_esperadas(capacidade, dados_ano['Media'], dados_ano['StdDev'])
    return vendas_esperadas * LUCRO_POR_MIL_UNID

# --- ALGORITMO DE PROGRAMAÇÃO DINÂMICA (BACKWARD INDUCTION) ---

# Definindo o espaço de estados (Capacidades possíveis)
# Começa em 150 e pode crescer até cobrir a demanda máxima (aprox 400) + margem.
# Vamos assumir um teto razoável para evitar loops infinitos, ex: 600.
estados_possiveis = range(150, 650, 50)
anos = range(1, 11)

# Tabela para armazenar o Valor Ótimo (VPL acumulado) de cada estado em cada ano
# Estrutura: dp[ano][capacidade] = {'vpl': valor, 'acao': expansao_escolhida}
dp = {ano: {} for ano in range(1, 12)}

# 1. CONDIÇÃO DE CONTORNO (ANO 11 - Pós-Horizonte/Perpetuidade)
# No ano 11, assumimos que o fluxo se estabiliza. Valor = Fluxo Perpetuo / TMA
print("Calculando valores terminais (Perpetuidade)...")
for cap in estados_possiveis:
    # Usamos os parâmetros do ano 10 para a estabilidade
    lucro_estavel = calcular_lucro_operacional(cap, 9) # índice 9 é o ano 10
    valor_terminal = lucro_estavel / TMA
    dp[11][cap] = {'vpl': valor_terminal, 'acao': 0}

# 2. RECURSÃO (Do Ano 10 até o Ano 1)
print("Executando recursão backward...")
for ano in range(10, 0, -1):
    idx_ano = ano - 1
    for cap_atual in estados_possiveis:
        melhor_vpl = -float('inf')
        melhor_acao = 0
        
        # Testar todas as decisões de expansão possíveis
        for expansao, custo_investimento in OPCOES_EXPANSAO.items():
            cap_futura = cap_atual + expansao
            
            # Verificar se a capacidade futura é um estado válido
            if cap_futura in dp[ano + 1]:
                
                # O investimento ocorre no início do ano (hoje)
                # O aumento de capacidade só fica disponível no PRÓXIMO ano
                # Mas o lucro deste ano é baseado na capacidade ATUAL
                lucro_operacional = calcular_lucro_operacional(cap_atual, idx_ano)
                
                fluxo_caixa_ano = lucro_operacional - custo_investimento
                
                # VPL = Fluxo deste ano + VPL do próximo ano trazido a valor presente
                vpl_total = (fluxo_caixa_ano + dp[ano + 1][cap_futura]['vpl']) / (1 + TMA)
                
                # Se estamos no ano 1, não descontamos o fluxo do ano 1 (pois é "hoje" ou início do período),
                # ou descontamos se assumirmos que os fluxos ocorrem ao final do ano.
                # Padrão em projetos: Investimento no instante 0, fluxos no fim do ano 1.
                # O modelo do slide 20 parece descontar tudo. Vamos manter a consistência da fórmula padrão:
                # VPL_n = (Fluxo_n + VPL_n+1) / (1+i). Isso traz tudo para o inicio do ano n.
                
                if vpl_total > melhor_vpl:
                    melhor_vpl = vpl_total
                    melhor_acao = expansao
        
        dp[ano][cap_atual] = {'vpl': melhor_vpl, 'acao': melhor_acao}

# --- RECONSTRUÇÃO DO PLANO ÓTIMO (FORWARD PASS) ---

cap_atual = CAPACIDADE_INICIAL
plano_otimo = []

print("Reconstruindo o caminho ótimo...")
for ano in anos:
    dados_decisao = dp[ano].get(cap_atual)
    
    if not dados_decisao:
        break
        
    acao = dados_decisao['acao']
    vpl_neste_ponto = dados_decisao['vpl']
    
    # Calcular métricas para exibição
    investimento = OPCOES_EXPANSAO[acao]
    lucro_op = calcular_lucro_operacional(cap_atual, ano-1)
    
    plano_otimo.append({
        'Ano': ano,
        'Capacidade Atual': cap_atual,
        'Decisão Expansão': acao,
        'Investimento (M$)': investimento,
        'Lucro Operacional Esp (M$)': round(lucro_op, 2),
        'Capacidade Próx Ano': cap_atual + acao
    })
    
    # Atualiza capacidade para o próximo loop
    cap_atual += acao

# Criar DataFrame do resultado
df_resultado = pd.DataFrame(plano_otimo)

# Exibir Resultados
from IPython.display import display
print("\n--- PLANO ÓTIMO DE EXPANSÃO ---")
display(df_resultado)

valor_empresa = dp[1][CAPACIDADE_INICIAL]['vpl']
print(f"\nValor da Ciclo Motores S/A (VPL Total): US$ {valor_empresa:,.2f} Milhões")

Calculando valores terminais (Perpetuidade)...
Executando recursão backward...
Reconstruindo o caminho ótimo...

--- PLANO ÓTIMO DE EXPANSÃO ---


,Ano,Capacidade Atual,Decisão Expansão,Investimento (M$),Lucro Operacional Esp (M$),Capacidade Próx Ano
0,1,150,100,190,150.43,250
1,2,250,0,0,198.02,250
2,3,250,150,270,248.85,400
3,4,400,0,0,305.84,400
4,5,400,0,0,352.01,400
5,6,400,0,0,382.64,400
6,7,400,0,0,400.12,400
7,8,400,50,100,409.37,450
8,9,450,0,0,425.26,450
9,10,450,0,0,429.85,450



Valor da Ciclo Motores S/A (VPL Total): US$ 3,411.45 Milhões
